# Bayesian biodiversity: Geodata processing


- Sources: population density raster (.tif), road network (.shp) and site coordinates (.shp)
- Target output: Population and road density at 1, 10, 50 and 100km
- Target output should be appended to the site coordinates geodataframe 
- Find the values: rasterio.sample
- Aggregate: use the pygeoprocessing function. Simple scaling of the pixel_size
- Use shapely to buffer a pixel around each site point coordinate
- Then intersect this with the road density shapefile

In [32]:
import pygeoprocessing as pgp
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.enums import Resampling
import shapely
import numpy as np
import time
from rasterstats import zonal_stats
import json
from pyproj import Transformer

In [4]:
# Load black for formatting
import jupyter_black

jupyter_black.load()

# Adjust display settings for pandas
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

## Load coordinate data from PREDICTS

In [69]:
gdf_sites = gpd.read_file("../../data/PREDICTS/site_coord.shp")
gdf_sites.head()

,SSS,geometry
0,AD1_2001__Liow 1 1,POINT (103.77861 1.35194)
1,AD1_2001__Liow 1 2,POINT (103.80806 1.35472)
2,AD1_2001__Liow 1 3,POINT (103.81167 1.39472)
3,AD1_2001__Liow 1 4,POINT (103.78722 1.32694)
4,AD1_2001__Liow 1 5,POINT (103.80361 1.28278)


In [6]:
# Get a list of coordinates from geometry
coord_list = [(x, y) for x, y in zip(gdf_sites["geometry"].x, gdf_sites["geometry"].y)]

## Gridded Population of the World (GPW), v4

https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals-rev11

Unit: Number of people per square kilometer.

In [7]:
pop_density_path_2000 = "../../data/GPW/gpw_v4_2000_30_sec.tif"
# pop_density_path_2005 = ("../../data/GPW/gpw_v4_2005_30_sec.tif")
# pop_density_path_2010 = ("../../data/GPW/gpw_v4_2010_30_sec.tif")
# pop_density_path_2015 = ("../../data/GPW/gpw_v4_2015_30_sec.tif")
# pop_density_path_2020 = ("../../data/GPW/gpw_v4_2020_30_sec.tif")

In [46]:
pgp.get_raster_info(pop_density_path_2000)

{'file_list': ['../../data/GPW/gpw_v4_2000_30_sec.tif'],
 'projection_wkt': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]',
 'geotransform': (-180.0,
  0.00833333333333387,
  0.0,
  90.00000000001157,
  0.0,
  -0.00833333333333387),
 'pixel_size': (0.00833333333333387, -0.00833333333333387),
 'raster_size': (43200, 21600),
 'n_bands': 1,
 'nodata': [-3.4028230607370965e+38],
 'overviews': [],
 'block_size': [43200, 1],
 'bounding_box': [-180.0, -90.0, 180.00000000002314, 90.00000000001157],
 'datatype': 6,
 'numpy_type': numpy.float32}

### Method 1: Rasterio sample

Get data for the grid in which the coordinates fall. This can work for high resolution data, but might be inaccurate when downsampling the population density data and then matching pixels with the site coordinates.

In [14]:
start = time.time()

# Load the raster file
raster_data = rasterio.open("../../data/GPW/gpw_v4_2000_30_sec.tif")

# Find the value used for nodata
nodata_value = raster_data.nodatavals[0]

# Extract population density from the grids for each coordinate
pop_density = np.array(list(raster_data.sample(coord_list))).flatten()

# Set density to np.nan if it seems to be a nodata pixel
if nodata_value is not None:
    pop_density[np.isclose(pop_density, nodata_value, equal_nan=True)] = np.nan

# Append new column to the dataframe
gdf_sites_point = gdf_sites.copy()
gdf_sites_point["Pop_density_2000_1km"] = pop_density

end = time.time()
print(end - start)

2.707219123840332


In [15]:
gdf_sites_point

,SSS,geometry,Pop_density_2000_1km
0,AD1_2001__Liow 1 1,POINT (103.77861 1.35194),12423.108398
1,AD1_2001__Liow 1 2,POINT (103.80806 1.35472),0.000000
2,AD1_2001__Liow 1 3,POINT (103.81167 1.39472),1935.903442
3,AD1_2001__Liow 1 4,POINT (103.78722 1.32694),4328.725586
4,AD1_2001__Liow 1 5,POINT (103.80361 1.28278),10090.784180
...,...,...,...
35731,YY1_2018__Guillemot 1 66,POINT (75.48022 12.19213),590.265381
35732,YY1_2018__Guillemot 1 67,POINT (75.48012 12.21315),590.265320
35733,YY1_2018__Guillemot 1 7,POINT (75.52462 12.26442),115.935951
35734,YY1_2018__Guillemot 1 8,POINT (75.52417 12.26392),115.935951


### Method 2: Reprojection + rasterstats zonal statistics

Buffer a square around each coordinate, and count the weighted average density under each square, using rasterstats

In [91]:
def project_and_buffer(initial_point, buffer_dist):

    # Get the coordinate point values
    long, lat = initial_point.x, initial_point.y

    # Determine the UTM zone and hemisphere
    zone_number = int((long + 180) // 6) + 1
    epsg_code = f"EPSG:{32700 + zone_number if lat < 0 else 32600 + zone_number}"

    # Initialize Transformer object, and transform to UTM coordinates
    utm_transformer = Transformer.from_crs("EPSG:4326", epsg_code, always_xy=True)
    utm_x, utm_y = utm_transformer.transform(long, lat)
    utm_coords = shapely.Point(utm_x, utm_y)

    # Buffer the Point into a square Polygon
    utm_coords_buff = shapely.buffer(utm_coords, buffer_dist, cap_style="square")

    # Transformer for reprojecting to global coordinate system
    global_transformer = Transformer.from_crs(epsg_code, "EPSG:4326", always_xy=True)

    # Apply the transformation to each point in the buffered polygon
    def _transform_point(x, y):
        global_x, global_y = global_transformer.transform(x, y)
        return global_x, global_y

    global_polygon = shapely.transform(_transform_point, utm_coords_buff)

    return global_polygon

In [86]:
start = time.time()

# Make a copy of the original dataframe
gdf_sites_buff = gdf_sites.copy()

# Set buffer distances
# buffer_dist_m = [1000, 10000, 25000, 50000]
buffer_dist = 1000

# Perform the buffering to square polygons
gdf_sites_buff["geometry"] = gdf_sites_buff.apply(
    lambda row: project_and_buffer(row["geometry"], buffer_dist), axis=1
)

# Save the file
gdf_sites_buff.to_file("../../data/PREDICTS/site_coord_buffered.shp")

end = time.time()
print(end - start)

136.32296109199524


In [87]:
start = time.time()

# Calculate zonal statistics
stats = zonal_stats(
    vectors="../../data/PREDICTS/site_coord_buffered.shp",
    raster="../../data/GPW/gpw_v4_2000_30_sec.tif",
    stats="mean",
    all_touched=True,
)

# Get values and add columns to original dataframe
pop_density = [x["mean"] for x in stats]
gdf_sites_buff = gdf_sites_buff.copy()
gdf_sites_buff["Pop_density_2000_1km"] = pop_density

end = time.time()
print(end - start)

89.63635611534119


In [88]:
gdf_sites_buff

,SSS,geometry,Pop_density_2000_1km
0,AD1_2001__Liow 1 1,"POLYGON ((103.78759 1.36099, 103.78760 1.34290...",8665.053819
1,AD1_2001__Liow 1 2,"POLYGON ((103.81704 1.36377, 103.81705 1.34568...",1131.090088
2,AD1_2001__Liow 1 3,"POLYGON ((103.82065 1.40377, 103.82066 1.38568...",4979.729167
3,AD1_2001__Liow 1 4,"POLYGON ((103.79620 1.33599, 103.79621 1.31790...",4332.883681
4,AD1_2001__Liow 1 5,"POLYGON ((103.81259 1.29183, 103.81260 1.27374...",8465.158203
...,...,...,...
35731,YY1_2018__Guillemot 1 66,"POLYGON ((75.48943 12.20116, 75.48940 12.18307...",590.265340
35732,YY1_2018__Guillemot 1 67,"POLYGON ((75.48933 12.22218, 75.48930 12.20409...",590.265299
35733,YY1_2018__Guillemot 1 7,"POLYGON ((75.53383 12.27345, 75.53380 12.25536...",193.610229
35734,YY1_2018__Guillemot 1 8,"POLYGON ((75.53338 12.27295, 75.53335 12.25486...",193.610229


## Global Roads Open Access Data Set (gROADS), v1

https://sedac.ciesin.columbia.edu/data/set/groads-global-roads-open-access-v1

One data set available for each continent.

### Metadata description

- `OBJECTID`: Object ID
- `SourceID`: Source ID
- `Picture`: Picture
- `Exs`: Existence Category. Options include 1=Definite, 2=Doubtful, 0=Unspecified
- `Notes`: Notes
- `RoadID`: Road ID
- `ONme`: Official Road Name
- `RteNme`: Route Name
- `NtlClass`: National Inventory Road Class
- `FClass`: Functional Class with options 1=Highway, 2=Primary, 3=Secondary, 4=Tertiary, 5=Local/ Urban, 6=Trail, 7=Private, 0=Unspecified
- `Crgway`: Carriageways. Options include 1=Single, 2=Dual, 0=Unspecified
- `NumLanes`: Number of lanes
- `LneWidthM`: Lane Width in meters
- `RdWidthM`: Road Width in meters
- `AxleLoadMT`: Maximum Axle Loading in MT
- `TotLoadMT`: Maximum Total Loading in MT
- `SrfTpe`: Surface Type with options 1=Paved, 2=Gravel, 3=Dirt/Sand, 4=Steel, 5=Wood, 6=Grass, 0=Unspecified
- `SrfCond`: Surface Condition with options 1=Rough (<40kph), 2=Smooth (>40kph), 3=Snow/Ice, 4=Mud, 0=Unspecified
- `SrfPrep`: Surface Preparation with options 1=Natural Compaction, 2=Traffic Compaction, 3=Engineered Compaction, 4=Uncompacted, 0=Unspecified
- `IsSeasonal`: Affected by Season. Options include 1=Yes, 2=No, 0=Unspecified
- `CurntPrac`: Current Road Practicability with options 1=Non-motorized, 2=Motorbike, 3=4WD <3.5MT, 4=Light Truck <10MT, 5=Heavy Truck <20MT, 6=Truck + Trailer >20MT, 0=Unspecified
- `GdWthrPrac`: Good Weather Road Practicability with options 1=Non-motorized, 2=Motorbike, 3=4WD <3.5MT, 4=Light Truck <10MT, 5=Heavy Truck <20MT, 6=Truck + Trailer >20MT, 0=Unspecified
- `BdWthrPrac`: Bad Weather Road Practicability. Options include 1=Non-motorized, 2=Motorbike, 3=4WD <3.5MT, 4=Light Truck <10MT, 5=Heavy Truck <20MT, 6=Truck + Trailer >20MT, 0=Unspecified
- `SpeedLimit`: Speed Limit in Km/hr
- `CurntSpeed`: Current Average Speed
- `GnralSpeed`: General Average Speed
- `IsUndrCstr`: Is under Construction / Repairs. Options include 1=Yes, 2=No, 0=Unspecified
- `CstWrkETC`: Construction Work Estimated Completion Date
- `GradDeg`: Gradient in degrees
- `Sec`: Road Security Category. Options include 1=Category A (low risk), 2=Category B (low to medium risk), 3=Category C (medium to high risk), 4=Category D (high risk), 5=Category E (critical risk), 0=Unspecified
- `HasShouldr`: Has Shoulder. Options include 1=Yes, 2=No, 0=Unspecified
- `HasSidewalk`: Has Sidewalk. Options include 1=Yes, 2=No, 0=Unspecified
- `DrivSide`: Driving Side. Options include 1=Left, 2=Right, 0=Unspecified
- `IsElevated`: Is elevated / suspended above ground/water. Options include 1=Yes, 2=No, 0=Unspecified
- `HasMedian`: Has Median. Options include 1=Yes, 2=No, 0=Unspecified
- `OpStatus`: Operational Status. Options include 1=Open, 2=Restricted, 3=Closed, 4=Abandoned/Disused, 0=Unspecified
- `Shape_Length`: Length of segment
- `Length_KM`: Length of segment in kilometers



In [93]:
roads_americas = "../../data/gROADS/americas/groads-v1-americas.shp"
# groads_africa =
# groads_asia =
# groads_europe =
# groads_oceania_east =
# groads_oceania_west =

In [94]:
gdf_roads = gpd.read_file(roads_americas)
gdf_roads.head(2)

,SOURCEID,EXS,NOTES,ROADID,ONME,RTENME,NTLCLASS,FCLASS,CRGWAY,NUMLANES,LNEWIDTHM,RDWIDTHM,AXLELOADMT,TOTLOADMT,SRFTPE,SRFCOND,SRFPREP,ISSEASONAL,CURNTPRAC,GDWTHRPRAC,BDWTHRPRAC,SPEEDLIMIT,CURNTSPEED,GNRALSPEED,ISUNDRCSTR,CSTWRKETC,GRADDEG,SEC,HASSHOULDR,HASSIDEWLK,DRIVSIDE,ISELEVATED,HASMEDIAN,OPSTATUS,LENGTH_KM,Shape_Leng,geometry
0,s034_0001,0.0,None,ITOS156229,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.086124,0.216327,"LINESTRING (-46.64422 -22.52663, -46.63456 -22..."
1,s034_0001,0.0,None,ITOS156230,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.525002,0.338829,"LINESTRING (-49.61825 -23.06787, -49.54568 -23..."


In [95]:
print(gdf_roads.crs)

EPSG:4326


## WorldClim elevation data

https://worldclim.org/data/worldclim21.html#google_vignette

Same structure and resolution as the population density data. Can use the same approach to extract values for buffered sampling locations.

In [89]:
elevation_path = "../../data/WorldClim/Elevation/wc2.1_30s_elev.tif"
pgp.get_raster_info(elevation_path)

{'file_list': ['../../data/WorldClim/Elevation/wc2.1_30s_elev.tif'],
 'projection_wkt': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]',
 'geotransform': (-180.0,
  0.008333333333333333,
  0.0,
  90.0,
  0.0,
  -0.008333333333333333),
 'pixel_size': (0.008333333333333333, -0.008333333333333333),
 'raster_size': (43200, 21600),
 'n_bands': 1,
 'nodata': [-32768.0],
 'overviews': [],
 'block_size': [43200, 1],
 'bounding_box': [-180.0, -90.0, 180.0, 90.0],
 'datatype': 3,
 'numpy_type': numpy.int16}

## WorldClim bioclimatic data

https://worldclim.org/data/bioclim.html

In [96]:
bioclim_path = "../../data/WorldClim/Bioclimatic/wc2.1_30s_bio_1.tif"
pgp.get_raster_info(bioclim_path)

{'file_list': ['../../data/WorldClim/Bioclimatic/wc2.1_30s_bio_1.tif'],
 'projection_wkt': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]',
 'geotransform': (-180.0,
  0.008333333333333333,
  0.0,
  90.0,
  0.0,
  -0.008333333333333333),
 'pixel_size': (0.008333333333333333, -0.008333333333333333),
 'raster_size': (43200, 21600),
 'n_bands': 1,
 'nodata': [-3.3999999521443642e+38],
 'overviews': [],
 'block_size': [43200, 1],
 'bounding_box': [-180.0, -90.0, 180.0, 90.0],
 'datatype': 6,
 'numpy_type': numpy.float32}

## Unused code

In [ ]:
downscale_factor = 10  # Go from 1km to 10km

with rasterio.open("../../data/GPW/gpw_v4_2000_30_sec.tif") as dataset:

    # Read dataset and specify output shape
    # Use the average population density of the pixels being aggregated
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height / downscale_factor),
            int(dataset.width / downscale_factor),
        ),
        resampling=Resampling.average,
    )

    # Scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]), (dataset.height / data.shape[-2])
    )

    # Define the metadata for the new downsampled raster
    out_meta = dataset.meta.copy()
    out_meta.update(
        {
            "driver": "GTiff",
            "height": data.shape[1],
            "width": data.shape[2],
            "transform": transform,
        }
    )

    # Write the downsampled raster to a new file
    with rasterio.open("../../data/GPW/gpw_v4_2000_10_km.tif", "w", **out_meta) as dest:
        dest.write(data)

In [ ]:
start = time.time()

# Load the raster file
raster_data = rasterio.open("../../data/GPW/gpw_v4_2000_10_km.tif")

# Find the value used for nodata
nodata_value = raster_data.nodatavals[0]

# Extract population density from the grids for each coordinate
pop_density = np.array(list(raster_data.sample(coord_list))).flatten()

# Set density to np.nan if it seems to be a nodata pixel
if nodata_value is not None:
    pop_density[np.isclose(pop_density, nodata_value, equal_nan=True)] = np.nan

# Append new column to the dataframe
gdf_sites_point = gdf_sites_point.copy()
gdf_sites_point["Pop_density_2000_10km"] = pop_density

end = time.time()
print(end - start)